# Data Preprocessing

In [16]:
# Import the Libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

#the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [17]:
# Preprocessing Data
num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

#Convert class vectors to the binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# Create the Model
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(28, kernel_size = (3, 3), input_shape = input_shape))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))
model.compile(loss = keras.losses.categorical_crossentropy, optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# Train the Model
hist = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, verbose = 1, validation_data = (x_test, y_test))
print("The model has Successfully Trained!!!!")
model.save('mnist_tranied_model.h5')
print("Saved the model as mnist_tranied_model_New.h5")

In [ ]:
# Evaluate the Model
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])

# Create the GUI and App

In [2]:
# Import the libraries to create GUI
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [19]:
model = load_model('mnist_tranied_model.h5')
def predict_digit(img):

    #resize image to 28x28 pixels
    img = img.resize((28, 28))
    
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    
    #Reshaping to support our model input and noarmalizing
    img = img.reshape(1, 28, 28, 1)
    img = img/255.0
    
    #Predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        
        #Creating Elements
        self.canvas = tk.Canvas(self, width = 900, height = 700, bg = "white", cursor = "cross")
        self.label = tk.Label(self, text = "Initializing", bg = "yellow", font = ("Helvetica", 30))
        self.classify_btn = tk.Button(self, text = "Recongise the Digit", bg = "green", font = ("Helvetica", 15), command = self.classify_handwriting)
        self.button_clear = tk.Button(self, text = "Clear", bg = "red", font = ("Helvetica", 15), command = self.clear_all)
        
        #Grid Structure
        self.canvas.grid(row = 0, column = 0, pady = 2, sticky = W )
        self.label.grid(row = 0, column = 1, pady = 2, padx = 2)
        self.classify_btn.grid(row = 1, column = 1, pady = 10, padx = 10)
        self.button_clear.grid(row = 1, column = 0, pady = 10)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() #get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) #get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text = 'Digit :' + str(digit) + '\n\n' + 'Accuracy: ' + str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 5
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill = 'black')
        
app = App()
mainloop()
          

1/1 [==============================] - 0s 16ms/step
